In [3]:
import pandas as pd
import plotly.express as px
import numpy as np
import json
import re

In [4]:
df = pd.read_csv('./data/electric_vehicle_population_data.csv')
df

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,JTMAB3FV3P,Kitsap,Seabeck,WA,98380.0,2023,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,42.0,0.0,35.0,240684006,POINT (-122.8728334 47.5798304),PUGET SOUND ENERGY INC,5.303509e+10
1,1N4AZ1CP6J,Kitsap,Bremerton,WA,98312.0,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151.0,0.0,35.0,474183811,POINT (-122.6961203 47.5759584),PUGET SOUND ENERGY INC,5.303508e+10
2,5YJ3E1EA4L,King,Seattle,WA,98101.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266.0,0.0,43.0,113120017,POINT (-122.3340795 47.6099315),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,1N4AZ0CP8E,King,Seattle,WA,98125.0,2014,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,108188713,POINT (-122.304356 47.715668),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
4,1G1FX6S00H,Thurston,Yelm,WA,98597.0,2017,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238.0,0.0,20.0,176448940,POINT (-122.5715761 46.9095798),PUGET SOUND ENERGY INC,5.306701e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205434,1N4BZ1CV2N,Chelan,Peshastin,WA,98847.0,2022,NISSAN,LEAF,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,12.0,187384494,POINT (-120.6051696 47.5510173),PUD NO 1 OF CHELAN COUNTY,5.300796e+10
205435,1FTVW1EV0P,Snohomish,Everett,WA,98208.0,2023,FORD,F-150,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,44.0,255036386,POINT (-122.2032349 47.8956271),PUGET SOUND ENERGY INC,5.306104e+10
205436,5YJXCDE22H,Spokane,Cheney,WA,99004.0,2017,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,200.0,0.0,6.0,221631588,POINT (-117.5836098 47.4951312),BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...,5.306301e+10
205437,5YJ3E1EA3J,King,Vashon,WA,98070.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215.0,0.0,34.0,336983496,POINT (-122.466938 47.429244),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10


# Exploratory Data Analysis (EDA)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205439 entries, 0 to 205438
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         205439 non-null  object 
 1   County                                             205436 non-null  object 
 2   City                                               205436 non-null  object 
 3   State                                              205439 non-null  object 
 4   Postal Code                                        205436 non-null  float64
 5   Model Year                                         205439 non-null  int64  
 6   Make                                               205439 non-null  object 
 7   Model                                              205438 non-null  object 
 8   Electric Vehicle Type                              205439 non-null  object

In [6]:
df.isnull().sum()

VIN (1-10)                                             0
County                                                 3
City                                                   3
State                                                  0
Postal Code                                            3
Model Year                                             0
Make                                                   0
Model                                                  1
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         8
Base MSRP                                              8
Legislative District                                 442
DOL Vehicle ID                                         0
Vehicle Location                                       8
Electric Utility                                       3
2020 Census Tract                                      3
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
df.describe()

,Postal Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,2020 Census Tract
count,205436.000000,205439.000000,205431.000000,205431.000000,204997.000000,2.054390e+05,2.054360e+05
mean,98177.971870,2020.960363,52.164342,922.670532,28.970848,2.277156e+08,5.297704e+10
std,2419.037479,2.989059,88.075859,7761.753602,14.910052,7.205737e+07,1.588435e+09
min,1731.000000,1997.000000,0.000000,0.000000,1.000000,4.469000e+03,1.001020e+09
25%,98052.000000,2019.000000,0.000000,0.000000,17.000000,1.935324e+08,5.303301e+10
50%,98125.000000,2022.000000,0.000000,0.000000,33.000000,2.382368e+08,5.303303e+10
75%,98372.000000,2023.000000,48.000000,0.000000,42.000000,2.618718e+08,5.305307e+10
max,99577.000000,2025.000000,337.000000,845000.000000,49.000000,4.792548e+08,5.602100e+10


In [9]:
df.describe(include=['object'])

,VIN (1-10),County,City,State,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Vehicle Location,Electric Utility
count,205439,205436,205436,205439,205439,205438,205439,205439,205431,205436
unique,12140,205,770,45,42,152,2,3,928,74
top,7SAYGDEE6P,King,Seattle,WA,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,POINT (-122.1207376 47.6705374),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)
freq,1221,105237,33328,204997,90318,43437,161539,114172,5203,75156


In [10]:
df['Model Year'].value_counts()

Model Year
2023    60161
2024    30030
2022    28465
2021    19837
2018    14386
2020    12241
2019    10872
2017     8662
2016     5474
2015     4741
2013     4331
2014     3507
2012     1549
2011      707
2025      415
2010       24
2008       22
2000        7
1999        4
2002        2
2003        1
1997        1
Name: count, dtype: int64

In [11]:
df['Make'].value_counts()

Make
TESLA                   90318
CHEVROLET               15114
NISSAN                  14525
FORD                    10840
KIA                      9104
BMW                      8481
TOYOTA                   7945
HYUNDAI                  5782
RIVIAN                   5679
VOLKSWAGEN               5638
JEEP                     5413
VOLVO                    4877
AUDI                     3918
CHRYSLER                 3693
MERCEDES-BENZ            1930
SUBARU                   1238
PORSCHE                  1232
POLESTAR                 1162
HONDA                    1078
MITSUBISHI               1018
MINI                     1015
FIAT                      799
LEXUS                     754
CADILLAC                  745
MAZDA                     711
DODGE                     665
LINCOLN                   299
LUCID                     289
GENESIS                   266
SMART                     248
JAGUAR                    243
FISKER                    186
ALFA ROMEO                 77
LAND 

In [12]:
df['Make'].nunique()

42

In [13]:
df['Model'].nunique()

152

In [14]:
df['Base MSRP'].value_counts()

Base MSRP
0.0         202114
69900.0       1359
31950.0        365
52900.0        220
32250.0        138
59900.0        128
54950.0        124
39995.0        116
36900.0        103
44100.0         99
64950.0         86
45600.0         81
52650.0         72
33950.0         71
34995.0         64
55700.0         54
36800.0         52
53400.0         39
81100.0         23
110950.0        21
98950.0         21
90700.0         16
102000.0        13
75095.0         11
184400.0        10
43700.0          9
89100.0          8
109000.0         6
91250.0          4
32995.0          3
845000.0         1
Name: count, dtype: int64

# Data Transformation

In [15]:
df = df.copy()
df['Vehicle_Latitude_Loc'] = df['Vehicle Location'].str.extract(r'^POINT \([0-9.-]+ ([0-9.-]+)\)$').astype(float)
df['Vehicle_Longitude_Loc'] = df['Vehicle Location'].str.extract(r'^POINT \(([0-9.-]+) [0-9.-]+\)$').astype(float)
df[['Vehicle_Latitude_Loc', 'Vehicle_Longitude_Loc']]

,Vehicle_Latitude_Loc,Vehicle_Longitude_Loc
0,47.579830,-122.872833
1,47.575958,-122.696120
2,47.609932,-122.334080
3,47.715668,-122.304356
4,46.909580,-122.571576
...,...,...
205434,47.551017,-120.605170
205435,47.895627,-122.203235
205436,47.495131,-117.583610
205437,47.429244,-122.466938


In [16]:
df['Make'] = df['Make'].apply(lambda x: x.title())
df['Make']

0            Toyota
1            Nissan
2             Tesla
3            Nissan
4         Chevrolet
            ...    
205434       Nissan
205435         Ford
205436        Tesla
205437        Tesla
205438        Tesla
Name: Make, Length: 205439, dtype: object

In [17]:
df['Make'] = df['Make'].where(df['Make'].isin(df['Make'].value_counts().nlargest(10).index.tolist()), 'Others')
df['Make'].value_counts()

Make
Tesla         90318
Others        32013
Chevrolet     15114
Nissan        14525
Ford          10840
Kia            9104
Bmw            8481
Toyota         7945
Hyundai        5782
Rivian         5679
Volkswagen     5638
Name: count, dtype: int64

In [18]:
top_producers = df.groupby(['Make', 'Electric Vehicle Type'])['Model'].count().reset_index()
top_producers.columns = ['Make', 'Electric Vehicle Type', 'Count']
top_producers = top_producers[top_producers['Make'] != 'Others']
top_producers

,Make,Electric Vehicle Type,Count
0,Bmw,Battery Electric Vehicle (BEV),3160
1,Bmw,Plug-in Hybrid Electric Vehicle (PHEV),5321
2,Chevrolet,Battery Electric Vehicle (BEV),10285
3,Chevrolet,Plug-in Hybrid Electric Vehicle (PHEV),4829
4,Ford,Battery Electric Vehicle (BEV),7124
5,Ford,Plug-in Hybrid Electric Vehicle (PHEV),3716
6,Hyundai,Battery Electric Vehicle (BEV),4904
7,Hyundai,Plug-in Hybrid Electric Vehicle (PHEV),878
8,Kia,Battery Electric Vehicle (BEV),6378
9,Kia,Plug-in Hybrid Electric Vehicle (PHEV),2726


In [31]:
df['Make'].value_counts()

Make
Tesla         90318
Others        32013
Chevrolet     15114
Nissan        14525
Ford          10840
Kia            9104
Bmw            8481
Toyota         7945
Hyundai        5782
Rivian         5679
Volkswagen     5638
Name: count, dtype: int64

In [30]:
df['Make'].value_counts().sum()

205439

In [20]:
line_top_producers = df[df['Make'] != 'Others']['Make'].value_counts().nlargest(3).index.tolist()
make_year_df = df[df['Make'].isin(line_top_producers)].groupby(['Make', 'Model Year'])['Model'].count().reset_index()
make_year_df.columns = ['Make', 'Model Year', 'Count']
make_year_df

,Make,Model Year,Count
0,Chevrolet,1997,1
1,Chevrolet,2011,70
2,Chevrolet,2012,452
3,Chevrolet,2013,777
4,Chevrolet,2014,732
5,Chevrolet,2015,464
6,Chevrolet,2016,310
7,Chevrolet,2017,2891
8,Chevrolet,2018,1267
9,Chevrolet,2019,1221


In [21]:
erange_production_count = df.groupby('Electric Range')['Model'].count().reset_index()
erange_production_count.columns = ['Electric Range', 'Count']
erange_df = pd.merge(df, erange_production_count, on='Electric Range')
erange_df = erange_df[(erange_df['Electric Range'] != 0) & (erange_df['Base MSRP'] != 0)]
erange_df

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,Vehicle_Latitude_Loc,Vehicle_Longitude_Loc,Count
12,KNDJX3AE8G,Yakima,Yakima,WA,98908.0,2016,Kia,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93.0,31950.0,14.0,169289970,POINT (-120.611068 46.596645),PACIFICORP,5.307700e+10,46.596645,-120.611068,523
26,5YJSA1DP4D,Yakima,Zillah,WA,98953.0,2013,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,15.0,161861780,POINT (-120.2658133 46.4063477),PACIFICORP,5.307700e+10,46.406348,-120.265813,2390
43,5YJSA1H10E,Kitsap,Port Orchard,WA,98366.0,2014,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,26.0,127169892,POINT (-122.6064912 47.5236145),PUGET SOUND ENERGY INC,5.303509e+10,47.523615,-122.606491,2390
135,5YJSA1H27E,Chelan,Leavenworth,WA,98826.0,2014,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,12.0,190374691,POINT (-120.6860209 47.7095564),PUD NO 1 OF CHELAN COUNTY,5.300796e+10,47.709556,-120.686021,2390
180,5YJSA1H10E,Snohomish,Bothell,WA,98012.0,2014,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,1.0,192953134,POINT (-122.206146 47.839957),PUGET SOUND ENERGY INC,5.306105e+10,47.839957,-122.206146,2390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205226,KNDJX3AE3G,King,Bellevue,WA,98004.0,2016,Kia,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93.0,31950.0,41.0,6009664,POINT (-122.202397 47.619252),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10,47.619252,-122.202397,523
205270,WBA7J2C36H,Pierce,South Hill,WA,98375.0,2017,Bmw,740E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14.0,89100.0,2.0,251461759,POINT (-122.321058 47.105259),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,5.305307e+10,47.105259,-122.321058,1116
205312,5YJSA1CP2D,Thurston,Olympia,WA,98516.0,2013,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,2.0,2703830,POINT (-122.78083 47.083975),PUGET SOUND ENERGY INC,5.306701e+10,47.083975,-122.780830,2390
205316,2C4RC1N75K,King,Redmond,WA,98053.0,2019,Others,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,32.0,39995.0,45.0,249136072,POINT (-122.030684 47.669552),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10,47.669552,-122.030684,4895


In [22]:
df[(df['Electric Range'] != 0) & (df['Base MSRP'] != 0)]

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,Vehicle_Latitude_Loc,Vehicle_Longitude_Loc
12,KNDJX3AE8G,Yakima,Yakima,WA,98908.0,2016,Kia,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93.0,31950.0,14.0,169289970,POINT (-120.611068 46.596645),PACIFICORP,5.307700e+10,46.596645,-120.611068
26,5YJSA1DP4D,Yakima,Zillah,WA,98953.0,2013,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,15.0,161861780,POINT (-120.2658133 46.4063477),PACIFICORP,5.307700e+10,46.406348,-120.265813
43,5YJSA1H10E,Kitsap,Port Orchard,WA,98366.0,2014,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,26.0,127169892,POINT (-122.6064912 47.5236145),PUGET SOUND ENERGY INC,5.303509e+10,47.523615,-122.606491
135,5YJSA1H27E,Chelan,Leavenworth,WA,98826.0,2014,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,12.0,190374691,POINT (-120.6860209 47.7095564),PUD NO 1 OF CHELAN COUNTY,5.300796e+10,47.709556,-120.686021
180,5YJSA1H10E,Snohomish,Bothell,WA,98012.0,2014,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,1.0,192953134,POINT (-122.206146 47.839957),PUGET SOUND ENERGY INC,5.306105e+10,47.839957,-122.206146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205234,KNDJX3AE3G,King,Bellevue,WA,98004.0,2016,Kia,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93.0,31950.0,41.0,6009664,POINT (-122.202397 47.619252),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10,47.619252,-122.202397
205278,WBA7J2C36H,Pierce,South Hill,WA,98375.0,2017,Bmw,740E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14.0,89100.0,2.0,251461759,POINT (-122.321058 47.105259),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,5.305307e+10,47.105259,-122.321058
205320,5YJSA1CP2D,Thurston,Olympia,WA,98516.0,2013,Tesla,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,69900.0,2.0,2703830,POINT (-122.78083 47.083975),PUGET SOUND ENERGY INC,5.306701e+10,47.083975,-122.780830
205324,2C4RC1N75K,King,Redmond,WA,98053.0,2019,Others,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,32.0,39995.0,45.0,249136072,POINT (-122.030684 47.669552),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10,47.669552,-122.030684


In [23]:
#df[(df['Electric Range'] != 0)]
df[(df['Base MSRP'] != 0)]['Base MSRP'].value_counts()

Base MSRP
69900.0     1359
31950.0      365
52900.0      220
32250.0      138
59900.0      128
54950.0      124
39995.0      116
36900.0      103
44100.0       99
64950.0       86
45600.0       81
52650.0       72
33950.0       71
34995.0       64
55700.0       54
36800.0       52
53400.0       39
81100.0       23
98950.0       21
110950.0      21
90700.0       16
102000.0      13
75095.0       11
184400.0      10
43700.0        9
89100.0        8
109000.0       6
91250.0        4
32995.0        3
845000.0       1
Name: count, dtype: int64

In [24]:
erange_df[['Electric Range', 'Count']].drop_duplicates().sort_values(by='Count', ascending=False)

,Electric Range,Count
991,32.0,4895
1561,220.0,4004
425,19.0,2442
26,208.0,2390
30375,33.0,1599
540,14.0,1116
508,17.0,799
2795,16.0,782
12,93.0,523
1503,12.0,166


In [25]:
erange_df.describe()

,Postal Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,2020 Census Tract,Vehicle_Latitude_Loc,Vehicle_Longitude_Loc,Count
count,3316.000000,3317.00000,3317.000000,3317.000000,3307.000000,3.317000e+03,3.316000e+03,3315.000000,3315.000000,3317.000000
mean,98147.064234,2015.60808,121.623455,57143.542358,30.076504,2.096345e+08,5.299318e+10,47.460845,-122.026166,1561.338559
std,2782.880201,2.49381,90.241458,22772.943759,14.625840,9.720884e+07,1.222387e+09,0.798186,1.965618,1125.260941
min,20152.000000,2008.00000,12.000000,31950.000000,1.000000,8.981300e+04,6.013313e+09,30.558957,-124.264464,27.000000
25%,98052.000000,2013.00000,17.000000,39995.000000,19.000000,1.452309e+08,5.303301e+10,47.358111,-122.440196,523.000000
50%,98122.000000,2016.00000,93.000000,59900.000000,34.000000,1.968321e+08,5.303303e+10,47.614119,-122.287143,1116.000000
75%,98371.000000,2018.00000,208.000000,69900.000000,43.000000,2.560404e+08,5.305307e+10,47.721052,-122.136803,2390.000000
max,99403.000000,2020.00000,265.000000,845000.000000,49.000000,4.788591e+08,5.307700e+10,48.991061,-76.773785,4895.000000


In [26]:
county_popular_make = df.groupby('County')['Make'].agg(lambda x: x.mode()[0]).reset_index()
county_popular_make

,County,Make
0,Ada,Tesla
1,Adams,Tesla
2,Alameda,Tesla
3,Albemarle,Chevrolet
4,Alexandria,Tesla
...,...,...
200,Williamson,Tesla
201,Yakima,Tesla
202,Yolo,Hyundai
203,York,Tesla


In [27]:
top_producers_ordering = df.groupby(['Make'])['Model'].count().reset_index()
top_producers_ordering = top_producers_ordering.rename(columns={'Model': 'Count'})
top_producers_ordering

,Make,Count
0,Bmw,8481
1,Chevrolet,15114
2,Ford,10840
3,Hyundai,5782
4,Kia,9104
5,Nissan,14525
6,Others,32012
7,Rivian,5679
8,Tesla,90318
9,Toyota,7945


In [28]:
df.describe()

,Postal Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,2020 Census Tract,Vehicle_Latitude_Loc,Vehicle_Longitude_Loc
count,205436.000000,205439.000000,205431.000000,205431.000000,204997.000000,2.054390e+05,2.054360e+05,205431.000000,205431.000000
mean,98177.971870,2020.960363,52.164342,922.670532,28.970848,2.277156e+08,5.297704e+10,47.437735,-122.021757
std,2419.037479,2.989059,88.075859,7761.753602,14.910052,7.205737e+07,1.588435e+09,0.814471,1.796891
min,1731.000000,1997.000000,0.000000,0.000000,1.000000,4.469000e+03,1.001020e+09,20.735185,-159.712613
25%,98052.000000,2019.000000,0.000000,0.000000,17.000000,1.935324e+08,5.303301e+10,47.358111,-122.395519
50%,98125.000000,2022.000000,0.000000,0.000000,33.000000,2.382368e+08,5.303303e+10,47.610347,-122.275332
75%,98372.000000,2023.000000,48.000000,0.000000,42.000000,2.618718e+08,5.305307e+10,47.721052,-122.136803
max,99577.000000,2025.000000,337.000000,845000.000000,49.000000,4.792548e+08,5.602100e+10,61.318822,-70.743216


# Data Exporting

In [29]:
df.to_csv('./assets/data/electric_vehicle_pop_clean.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'assets\data'